In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ====================== PHẦN TIỀN XỬ LÝ DỮ LIỆU GỐC ======================
# Đọc dữ liệu
print("Đang đọc dữ liệu...")
df = pd.read_csv("/kaggle/input/usa-real-estate-dataset/realtor-data.zip.csv")  

# Kiểm tra thông tin và giá trị null
print("Thông tin dữ liệu gốc:")
df.info()
print("\nSố lượng giá trị null:")
print(((df.isna().sum() / len(df)) * 100).sort_values(ascending=False))

# Loại bỏ các cột không cần thiết
df = df.drop(["city", "zip_code", "prev_sold_date"], axis=1)

# Xử lý giá trị null
df = df.drop(df[df['price'].isnull()].index)
df = df[~(df.isna().sum(axis=1) >= 2)]
df = df.drop(df[df['bed'].isnull()].index, axis=0)
df = df.drop(df[df['bath'].isnull()].index, axis=0)

# Xử lý cột status
print("\nGiá trị trong cột status:")
print(df["status"].value_counts())
df = df.drop("status", axis=1)

# Xử lý cột state
print("\nGiá trị trong cột state:")
print(df["state"].value_counts())
df = df.drop(df[df["state"].map(df["state"].value_counts()) < 50]["state"].index)

# Mã hóa state thành dạng số
label_encoder = LabelEncoder()
df['state_numeric'] = label_encoder.fit_transform(df['state'])
df = df.drop("state", axis=1)

# Tạo mapping từ số đến tên state
numeric_to_state = {label: state for label, state in enumerate(label_encoder.classes_)}
print("\nMapping state_numeric -> state:")
print(numeric_to_state)

# Mô tả dữ liệu
print("\nThống kê mô tả:")
print(df.describe())

Đang đọc dữ liệu...
Thông tin dữ liệu gốc:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2226382 entries, 0 to 2226381
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   brokered_by     float64
 1   status          object 
 2   price           float64
 3   bed             float64
 4   bath            float64
 5   acre_lot        float64
 6   street          float64
 7   city            object 
 8   state           object 
 9   zip_code        float64
 10  house_size      float64
 11  prev_sold_date  object 
dtypes: float64(8), object(4)
memory usage: 203.8+ MB

Số lượng giá trị null:
prev_sold_date    32.981627
house_size        25.533983
bath              22.986666
bed               21.618797
acre_lot          14.624130
street             0.488056
brokered_by        0.203604
price              0.069215
city               0.063197
zip_code           0.013430
state              0.000359
status             0.000000
dtype: float64

Giá trị 

In [ ]:
# Xử lý outlier cho cột price
q95, q25 = np.percentile(df["price"], [95, 25])
iqrMax = q95 + q25
print(f"\nThreshold cho price: {iqrMax}")
percent_outliers = len(df[df["price"] > 3150000.0]) / len(df["price"].index) * 100
print(f"Tỷ lệ outliers trong price: {percent_outliers:.2f}%")
df = df.drop(df[df["price"] > 3150000.0].index)

# Xử lý outlier cho acre_lot
percent_outliers = len(df[df["acre_lot"] > 200]) / len(df["acre_lot"].index) * 100
print(f"\nTỷ lệ outliers trong acre_lot: {percent_outliers:.2f}%")
df = df.drop(df[df["acre_lot"] > 200].index, axis=0)

# Xử lý outlier cho house_size
percent_outliers = len(df[df["house_size"] >= 20000]) / len(df["house_size"].index) * 100
print(f"\nTỷ lệ outliers trong house_size: {percent_outliers:.2f}%")
df = df.drop(df[df["house_size"] >= 20000].index, axis=0)

# Kiểm tra lại giá trị null
print("\nSố lượng giá trị null sau khi xử lý outliers:")
print(df.isna().sum().sort_values(ascending=False))


Threshold cho price: 1735000.0
Tỷ lệ outliers trong price: 1.47%

Tỷ lệ outliers trong acre_lot: 0.11%

Tỷ lệ outliers trong house_size: 0.02%

Số lượng giá trị null sau khi xử lý outliers:
acre_lot         237719
house_size        69812
street             3820
brokered_by        2196
bath                  0
bed                   0
price                 0
state_numeric         0
dtype: int64


In [ ]:

# Xử lý missing value cho acre_lot sử dụng XGBoost
print("\nĐang điền giá trị thiếu cho acre_lot bằng XGBoost...")

# Chuẩn bị dữ liệu
filled_df = df.dropna(subset=["acre_lot"])
missing_df = df[df['acre_lot'].isna()]

# Tạo X_train, y_train từ dữ liệu không có NaN
X_train_fill = filled_df.drop("acre_lot", axis=1)
y_train_fill = filled_df["acre_lot"]

# Tạo X_test từ dữ liệu cần điền
X_test_fill = missing_df.drop("acre_lot", axis=1)

# Xử lý dữ liệu để đảm bảo không có NaN trong dữ liệu huấn luyện
X_train_fill = X_train_fill.fillna(X_train_fill.mean())
X_test_fill = X_test_fill.fillna(X_train_fill.mean())

# Huấn luyện mô hình XGBoost để dự đoán acre_lot
model_fill = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist'
)
model_fill.fit(X_train_fill, y_train_fill)

# Dự đoán và điền giá trị thiếu
preds = model_fill.predict(X_test_fill)
df.loc[df['acre_lot'].isna(), 'acre_lot'] = preds

print(f"Đã điền {len(preds)} giá trị thiếu cho acre_lot")




Đang điền giá trị thiếu cho acre_lot bằng XGBoost...
Đã điền 237719 giá trị thiếu cho acre_lot


In [ ]:
# Xử lý missing value cho house_size bằng XGBoost
print("\nĐang điền giá trị thiếu cho house_size bằng XGBoost...")

# Chuẩn bị dữ liệu
filled_df = df.dropna(subset=["house_size"])
missing_df = df[df['house_size'].isna()]

# Tạo X_train, y_train từ dữ liệu không có NaN
X_train_fill = filled_df.drop("house_size", axis=1)
y_train_fill = filled_df["house_size"]

# Tạo X_test từ dữ liệu cần điền
X_test_fill = missing_df.drop("house_size", axis=1)

# Xử lý dữ liệu để đảm bảo không có NaN trong dữ liệu huấn luyện
X_train_fill = X_train_fill.fillna(X_train_fill.mean())
X_test_fill = X_test_fill.fillna(X_train_fill.mean())

# Huấn luyện mô hình XGBoost để dự đoán house_size
model_fill = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist'
)
model_fill.fit(X_train_fill, y_train_fill)

# Dự đoán và điền giá trị thiếu
preds = model_fill.predict(X_test_fill)
df.loc[df['house_size'].isna(), 'house_size'] = preds

print(f"Đã điền {len(preds)} giá trị thiếu cho house_size")

# Kiểm tra giá trị null sau khi xử lý
print("\nSố lượng giá trị null sau khi xử lý:")
print(df.isna().sum().sort_values(ascending=False))


Đang điền giá trị thiếu cho house_size bằng XGBoost...
Đã điền 69812 giá trị thiếu cho house_size

Số lượng giá trị null sau khi xử lý:
street           3820
brokered_by      2196
bed                 0
price               0
bath                0
acre_lot            0
house_size          0
state_numeric       0
dtype: int64


In [ ]:
# ====================== PHẦN CẢI TIẾN MÔ HÌNH ======================
# 1. Tạo đặc trưng mới - Feature Engineering
print("Đang tạo các đặc trưng mới...")

# Tạo đặc trưng mới từ các biến hiện có - GIỮ NGUYÊN THANG ĐO GỐC
print("Tạo các đặc trưng tương tác...")
df['bed_bath_ratio'] = df['bed'] / df['bath'].replace(0, 1)  # Tránh chia cho 0
df['total_rooms'] = df['bed'] + df['bath']
df['room_density'] = df['total_rooms'] / df['house_size'].replace(0, 1)
df['house_size_per_bed'] = df['house_size'] / df['bed'].replace(0, 1)
df['house_size_per_bath'] = df['house_size'] / df['bath'].replace(0, 1)
df['lot_to_house_ratio'] = df['acre_lot'] / df['house_size'].replace(0, 1)
df['size_by_state'] = df['house_size'] * df['state_numeric']
df['rooms_by_state'] = df['total_rooms'] * df['state_numeric']

# Loại bỏ bất kỳ giá trị NaN mới từ feature engineering
df = df.fillna(df.mean())



Đang tạo các đặc trưng mới...
Tạo các đặc trưng tương tác...


In [ ]:
# Chuẩn bị dữ liệu cho mô hình dự đoán - GIỮ THANG ĐO GỐC
print("\nChuẩn bị dữ liệu cho mô hình...")
X = df.drop(['price'], axis=1)
y = df['price']  # Target giữ nguyên thang đo gốc

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Kích thước tập huấn luyện: {X_train.shape}, tập kiểm tra: {X_test.shape}")




Chuẩn bị dữ liệu cho mô hình...
Kích thước tập huấn luyện: (1318375, 15), tập kiểm tra: (329594, 15)

Bắt đầu tìm kiếm siêu tham số tối ưu...


In [ ]:
# Thiết lập grid search cho XGBoost
print("\nBắt đầu tìm kiếm siêu tham số tối ưu...")
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0, 0.1, 0.3, 0.5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 1.0],
    'reg_lambda': [0, 0.1, 1.0]
}

# Sử dụng RandomizedSearchCV thay vì GridSearchCV để tiết kiệm thời gian
random_search = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(
        objective='reg:squarederror',
        tree_method='hist'
    ),
    param_distributions=param_grid,
    n_iter=30,  # Số lượng tổ hợp tham số cần thử
    scoring='r2',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Huấn luyện
random_search.fit(X_train, y_train)

# Lấy tham số tốt nhất
best_params = random_search.best_params_
print("Tham số tốt nhất:", best_params)
print(f"Best cross-validation score: {random_search.best_score_:.4f}")

# Huấn luyện mô hình với tham số tốt nhất
print("\nHuấn luyện mô hình cuối cùng với tham số tốt nhất...")
best_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    tree_method='hist',
    **best_params
)
best_model.fit(X_train, y_train)



Fitting 5 folds for each of 30 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
# Dự đoán
y_pred = best_model.predict(X_test)

# Đánh giá trên thang gốc
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nĐánh giá trên thang gốc (giá nhà thực tế):")
print(f"MAE: ${mae:.2f}")
print(f"MSE: ${mse:.2f}")
print(f"RMSE: ${rmse:.2f}")
print(f"R²: {r2:.4f}")



In [ ]:
# Đặt ở đầu notebook
%matplotlib inline

# Xóa bất kỳ biểu đồ đang tồn tại trước khi vẽ mới
plt.close('all')

# Khi vẽ biểu đồ
try:
    print("Đang vẽ biểu đồ...")
    
    # 1. Biểu đồ dự đoán vs thực tế
    plt.figure(figsize=(12, 10), clear=True)  # clear=True để xóa bất kỳ nội dung cũ
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'r--')
    plt.title('Giá trị dự đoán vs Thực tế', fontsize=15)
    plt.xlabel('Giá trị thực tế ($)', fontsize=12)
    plt.ylabel('Giá trị dự đoán ($)', fontsize=12)
    plt.grid(True)
    plt.show()
    plt.close()  # Đóng biểu đồ hiện tại trước khi vẽ tiếp theo
    
    # 2. Biểu đồ tầm quan trọng đặc trưng
    plt.figure(figsize=(12, 10), clear=True)
    feature_importance = best_model.feature_importances_
    feature_names = X.columns
    indices = np.argsort(feature_importance)[::-1]
    top_n = min(10, len(feature_names))
    
    plt.barh(range(top_n), feature_importance[indices][:top_n], align='center')
    plt.yticks(range(top_n), [feature_names[i] for i in indices][:top_n])
    plt.xlabel('Tầm quan trọng', fontsize=12)
    plt.title('Top 10 đặc trưng quan trọng nhất', fontsize=15)
    plt.tight_layout()
    plt.show()
    plt.close()
    
except Exception as e:
    print(f"Lỗi khi vẽ biểu đồ: {e}")
    import traceback
    traceback.print_exc()